In [2]:
import cv2
import matplotlib.pyplot as plt 
import numpy as np

In [20]:
# load image from the IAM database
url = 'aligned.png'
'''
image = Image.open(url).convert('L')
threshold = 127
image = image.point(lambda p: 255 if p > threshold else 0)
image = image.convert('1')
image
'''
img = cv2.imread(url, cv2.IMREAD_GRAYSCALE)
#otsu_threshold, image = cv2.threshold(
    #image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,
#)

# do adaptive threshold on gray image
img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 199, 15)

# make background of input white where thresh is white
#result = image.copy()

# write results to disk
cv2.imwrite("binary-aligned.png", img)
#cv2.imwrite("math_diagram_processed.jpg", result)

# display it
#cv2.imshow("THRESHOLD", thresh)
#cv2.imshow("RESULT", result)
#cv2.waitKey(0)

#plt.imshow(image, cmap='gray')

True

In [21]:
img.shape

(3736, 2048)

In [22]:
# Start by finding all of the connected components (white blobs in your image).
# 'im' needs to be grayscale and 8bit.
#img_uint8 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_uint8 = ((255-img)).astype(np.uint8)
print(img_uint8.shape)

num_components, img_with_separated_components, stats, _ = cv2.connectedComponentsWithStats(img_uint8)
# im_with_separated_blobs is an image where each detected blob has a different pixel value ranging from 1 to nb_blobs - 1.
# The background pixels have value 0.
print(num_components)
# 'stats' (and the silenced output 'centroids') provides information about the blobs. See the docs for more information. 
# Here, we're interested only in the size of the blobs :
sizes = stats[:, cv2.CC_STAT_AREA]
# You can also directly index the column with '-1' instead of 'cv2.CC_STAT_AREA' as it's the last column.

# A small gotcha is that the background is considered as a blob, and so its stats are included in the stats vector at position 0.

# minimum size of particles we want to keep (number of pixels).
# here, it's a fixed value, but you can set it as you want, eg the mean of the sizes or whatever.
min_size = 159  

# create empty output image with will contain only the biggest composents
img = np.zeros_like(img)

# for every component in the image, keep it only if it's above min_size.
# we start at 1 to avoid considering the background
for component in range(1, num_components):
    if sizes[component] >= min_size:
        img[img_with_separated_components == component] = 255


cv2.imwrite("denoised.png", img)

(3736, 2048)
387


True

In [23]:
kernel = np.ones((3, 3), np.uint8)
#img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=1)

kernel = np.ones((2, 2), np.uint8)
#img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=2)

kernel = np.ones((2, 1), np.uint8)
#img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=2)

kernel = np.ones((1, 2), np.uint8)
#img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=2)

kernel = np.ones((2, 1), np.uint8)
img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=3)

kernl = np.ones((2, 2), np.uint8)
#img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=3)

cv2.imwrite("denoised.png", img)

True

In [24]:
num_components, img_with_separated_components, stats, _ = cv2.connectedComponentsWithStats(img)

move_boxes = []
print(num_components)

for component in range(1, num_components):
    x = stats[component, cv2.CC_STAT_LEFT]
    y = stats[component, cv2.CC_STAT_TOP]
    w = stats[component, cv2.CC_STAT_WIDTH]
    h = stats[component, cv2.CC_STAT_HEIGHT]
    
    if 3. < w/h < 4.:
        move_boxes.append((component, x, y, w, h))
        

print(move_boxes)

# sort boxes into correct order

move_boxes.sort(key=lambda x: x[1])
move_boxes[:20] = sorted(move_boxes[:20], key=lambda x: x[2])
move_boxes[20:40] = sorted(move_boxes[20:40], key=lambda x: x[2])
move_boxes[40:60] = sorted(move_boxes[40:60], key=lambda x: x[2])
move_boxes[60:80] = sorted(move_boxes[60:80], key=lambda x: x[2])

test = img.copy()

temp = []
for i in range(20):
    temp.append(move_boxes[i])
    temp.append(move_boxes[i+20])
for i in range(40, 60):
    temp.append(move_boxes[i])
    temp.append(move_boxes[i+20])
move_boxes = temp

i = 0
for move_box in move_boxes:
    component, x, y, w, h = move_box

    cv2.putText(test,str(i), 
    (x, y+h), 
    cv2.FONT_HERSHEY_SIMPLEX, 
    1,
    127,
    1,
    2)
    i += 1

cv2.imwrite("denoised.png", test)


#cv2.imwrite("denoised.png", test)


278
[(45, 1168, 489, 436, 131), (47, 150, 490, 430, 132), (48, 588, 491, 435, 132), (55, 149, 629, 431, 133), (56, 588, 629, 435, 134), (58, 1168, 628, 437, 134), (64, 148, 768, 432, 134), (65, 1168, 769, 437, 134), (66, 588, 770, 436, 132), (71, 148, 907, 433, 136), (72, 588, 910, 437, 134), (74, 1168, 910, 437, 136), (83, 148, 1048, 434, 137), (84, 590, 1050, 435, 138), (86, 1168, 1053, 437, 136), (97, 149, 1189, 434, 138), (98, 590, 1193, 435, 137), (100, 1168, 1196, 436, 136), (101, 1611, 1197, 427, 140), (108, 149, 1331, 434, 139), (109, 590, 1336, 435, 137), (111, 1168, 1339, 434, 136), (112, 1609, 1340, 429, 139), (115, 149, 1474, 434, 139), (116, 590, 1479, 434, 135), (118, 1166, 1481, 434, 137), (119, 1606, 1484, 432, 137), (127, 150, 1617, 433, 138), (128, 591, 1621, 433, 135), (130, 1166, 1624, 432, 136), (131, 1605, 1627, 431, 135), (139, 150, 1759, 433, 137), (140, 590, 1763, 434, 135), (142, 1166, 1765, 431, 136), (143, 1605, 1768, 431, 136), (154, 151, 1901, 432, 136), (

IndexError: list index out of range

In [7]:
# ideas to deal with 'g', since it often spills over into the cell below
# 1. floodfill cells only from the TOP — this will filter out the bottom part of a stray 'g' from the cell above
# 2. crop out the bottom by set # of pixels as usual — this may turn our 'g' into a '9', so our NN should treat '9's as 'g's
# 3. maybe do an area-based noise reduction after cropping the bottom?

In [14]:
# pixel length to crop out from each side (ideally the maximum width of a grid line)
_, x, y, w, h = move_boxes[60]

crop = 5

test_box = 255 - img[y+crop:y+h-crop, x+crop:x+w-crop]   
cv2.imwrite("box.png", test_box)

True

In [15]:
# FIX THIS!!!!!!!!!!!
# Seems like this only pads by 1 pixel instead of 2?

def format_to_mnist(img):

    img = cv2.GaussianBlur(img,(5,5),1)

    img_h, img_w = img.shape
    dim_size_max = max(img.shape)

    if dim_size_max == img_w:
        im_h = (24 * img_h) // img_w
        if im_h <= 0 or img_w <= 0:
            print("Invalid Image Dimention: ", im_h, img_w, img_h)
        tmp_img = cv2.resize(img, (24,im_h),0,0,cv2.INTER_CUBIC)
    else:
        im_w = (24 * img_w) // img_h
        if im_w <= 0 or img_h <= 0:
            print("Invalid Image Dimention: ", im_w, img_w, img_h)
        tmp_img = cv2.resize(img, (im_w, 24),0,0,cv2.INTER_CUBIC)

    out_img = np.zeros((28, 28), dtype=np.ubyte)

    nb_h, nb_w = out_img.shape
    na_h, na_w = tmp_img.shape
    y_min = (nb_w) // 2 - (na_w // 2)
    y_max = y_min + na_w
    x_min = (nb_h) // 2 - (na_h // 2)
    x_max = x_min + na_h

    out_img[x_min:x_max, y_min:y_max] = tmp_img

    return out_img

In [16]:
# Find the contours
contours, hierarchy = cv2.findContours(test_box, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contours = sorted(contours, key=lambda x:cv2.boundingRect(x)[0])

# Loop over each contour and save it as a new image
for i, contour in enumerate(contours):
    # Create a mask for the current contour
    mask = np.zeros_like(test_box)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    
    # Extract the contour region from the original image using the mask
    contour_region = cv2.bitwise_and(test_box, test_box, mask=mask)
    
    # Find the bounding box of the contour to crop the region
    x, y, w, h = cv2.boundingRect(contour)
    cropped_contour = contour_region[y:y+h, x:x+w]

    # convert to MNIST format
    cropped_contour = format_to_mnist(cropped_contour)
    
    # Save the extracted contour region as a new image
    cv2.imwrite(f'mnist_images/contour_{i}.png', cropped_contour)